In [1]:
# MNIST
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# device config
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# hyper parameters
input_size=784    #  28*28
hidden_size=500
hidden_size1=200
num_classes=10
num_epochs=2
batch_size=100
learning_rate=0.001


#MNIST

train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())


train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)


examples=iter(train_loader)
samples,labels=examples.next()
print(samples.shape)
print(labels.shape)

for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(samples[i][0],cmap='gray')
    #plt.show()
    
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,hidden_size1,num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.l2=nn.Linear(hidden_size,hidden_size1)
        self.l3=nn.Linear(hidden_size1,num_classes)
        
        
        
    def forward(self,x):
        out=self.l1(x) #x - images of reshaped size 28*28 = 784, which should resemble input_size.
        out=self.relu(out)
        out=self.l2(out)
        out=self.l3(out)
        return out
        
model = NeuralNet(input_size,hidden_size,hidden_size1,num_classes).to(device)

#loss and optimizer
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


#trainining loop
n_total_steps=len(train_loader)

for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        #100,1,28,28
        #100,784
        images=images.reshape(-1,28*28).to(device) # Image is converted to a single vector of size
        labels=labels.to(device)
        
        #forward
        outputs=model.forward(images)
        loss=criterion(outputs,labels)
        
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+1)%100==0:
            print(f'epoch {epoch+1}/{num_epochs}, step{i+1}/{n_total_steps}, loss={loss.item():.4f}')
            
#test (we dont want to compute gradients for testing)
with torch.no_grad():
    n_correct=0#no of correct predictions.
    n_samples=0
    for images,labels in test_loader:
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        outputs=model.forward(images)
        
        #values, index
        _,predictions=torch.max(outputs.data,1)
        n_samples+=labels.size(0)
        n_correct+=(predictions==labels).sum().item()
    acc=100.0*n_correct/n_samples
    print(f'accuracy: {acc}')

torch.Size([100, 1, 28, 28])
torch.Size([100])
epoch 1/2, step100/600, loss=0.2260
epoch 1/2, step200/600, loss=0.2933
epoch 1/2, step300/600, loss=0.2447
epoch 1/2, step400/600, loss=0.4351
epoch 1/2, step500/600, loss=0.2059
epoch 1/2, step600/600, loss=0.1248
epoch 2/2, step100/600, loss=0.1332
epoch 2/2, step200/600, loss=0.0673
epoch 2/2, step300/600, loss=0.2088
epoch 2/2, step400/600, loss=0.0820
epoch 2/2, step500/600, loss=0.0338
epoch 2/2, step600/600, loss=0.1220
accuracy: 97.51
